We'll try to fetch the artists and tracks I listened to in a specific period from lastfm and see if we can enrich this info with spotify data (because it only allows to 50 last tracks in a beta api)

In [1]:
import requests as re
#import numpy as np
import time
import pandas as pd
import datetime
from datetime import date, timedelta
from IPython.core.display import clear_output
import json
import requests_cache

## Lastfm API - my played tracks API

In [2]:
#create a request cache
requests_cache.install_cache('get_recent_tracks_cache')

In [3]:
# define a function to generate the array for each response
def structure_table(_artist, _song, _mbid, _date):
    _arr  = [_artist, _song, _date]
    return _arr

#define an iteration fuction to work on each page, run the array generation function and append it to a final response array
def page_iterator(_response):
    for i in range(len(_response)):
        r = structure_table(_response[i]['artist']['#text'],_response[i]['name'], _response[i]['mbid'], _response[i]['date']['uts'])
        #append to the result array
        responses.append(r)

In [4]:
#Get the tracks played from 2020 until yesterday
start_date = datetime.date(2020,1,1)
_from = int(time.mktime(start_date.timetuple()))

responses = []
#Define header
headers = {'user-agent': 'my_played_tracks'}

#First request
params = {
        'method': 'user.getrecenttracks',
        'limit': 200,
        'api_key': 'cf786f78db52a45f40f6e7b573b7d211',
        'format': 'json',
        'user': 'ommarra',
        'extended': 0,
        'from':_from,
        'page': 1
        }

response = re.get('http://ws.audioscrobbler.com/2.0/', headers = headers, params = params)
response = response.json()['recenttracks']
page_iterator(response['track'])

#get the total pages
_total_pages = int(response['@attr']['totalPages'])
_page = int(response['@attr']['page']) + 1

In [5]:
#loop through all other pages
while _page <= _total_pages:
    #Just so we can keep an eye on it
    clear_output(wait=True)
    print("requesting page", _page, "from", _total_pages, "pages")

    params = {
        'method': 'user.getrecenttracks',
        'limit': 200,
        'api_key': 'cf786f78db52a45f40f6e7b573b7d211',
        'format': 'json',
        'user': 'ommarra',
        'extended': 0,
        'from': _from,
        'page': _page
        }

    response = re.get('http://ws.audioscrobbler.com/2.0/', headers = headers, params = params)
    
    if response.status_code == 200:
        response = response.json()['recenttracks']
        page_iterator(response['track'])
    else:
        print("error:", response.status_code)
        break
    
    # If not cached, sleep to keep the api requests per second low
    if not getattr(response, 'from_cache', False):
        time.sleep(0.2)

    #add pagination

    _page = int(response['@attr']['page']) + 1

requesting page 62 from 62 pages


In [6]:
#Saving the results to a csv file
pd.DataFrame(responses).to_csv("played_tracks.csv", index = False)